In [4]:

import tkinter as tk
from tkinter import *
import cv2
from PIL import Image, ImageTk
import os
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [5]:

emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
emotion_model.load_weights('model50.h5')

In [8]:

cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "   Angry   ", 1: "Disgusted", 2: "  Fearful  ", 3: "   Happy   ", 4: "  Natural  ", 5: "    Sad    ", 6: "Surprised"}
emoji_dist={0:"image/angry.png",1:"image/disgusted.png",2:"image/fearful.png",3:"image/happy.png",4:"image/neutral.png",5:"image/sad.png",6:"image/surpriced.png"}
global last_frame1                                    
last_frame1 = np.zeros((480, 640, 3), dtype=np.uint8)
global cap1

show_text=[0]
def show_vid():      
    cap1 = cv2.VideoCapture(0)
    if not cap1.isOpened():                             
        print("cant open the camera1")    
    flag1, frame1 = cap1.read()
    if frame1 is not None:
        print("frame1 is not none")
        frame1 = cv2.resize(frame1,dsize=(400,300))
        bounding_box = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
        print(num_faces)
        for (x, y, w, h) in num_faces:
            cv2.rectangle(frame1, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_gray_frame = gray_frame[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
            prediction = emotion_model.predict(cropped_img)
            maxindex = int(np.argmax(prediction))
            cv2.putText(frame1, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            show_text[0]=maxindex
            print(show_text[0],maxindex,"show_text")
   
    if flag1 is None:
        print ("Major error!")
    elif flag1:
        global last_frame1
        last_frame1 = frame1.copy()
        pic = cv2.cvtColor(last_frame1, cv2.COLOR_BGR2RGB)     
        img = Image.fromarray(pic)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        lmain.after(40, show_vid2)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit()
def show_vid2():
    print(emoji_dist[show_text[0]])
    frame2=cv2.imread(emoji_dist[show_text[0]])
    print(frame2)
    if frame2 is not None:
        print("frame2 is not none")
        pic2=cv2.cvtColor(frame2,cv2.COLOR_BGR2RGB)
        img2=Image.fromarray(frame2)
        imgtk2=ImageTk.PhotoImage(image=img2)
        lmain2.imgtk2=imgtk2
        lmain3.configure(text=emotion_dict[show_text[0]],font=('arial',45,'bold'))
        
        lmain2.configure(image=imgtk2,width=250, height=250)
        lmain2.after(40, show_vid)
class FullScreenApp(object):
    def __init__(self, master, **kwargs):
        self.master=master
        pad=3
        self._geom='200x200+0+0'
        master.geometry("{0}x{1}+0+0".format(
            master.winfo_screenwidth()-pad, master.winfo_screenheight()-pad))
        master.bind("<Return>",self.toggle_geom)            
    def toggle_geom(self,event):
        geom=self.master.winfo_geometry()
        print(geom,self._geom)
        self.master.geometry(self._geom)
        self._geom=geom
if __name__ == '__main__':
    root=tk.Tk() 
    # use global variable to store the image
    global img

    img = ImageTk.PhotoImage(Image.open("img4.jpg"))
    # keep the reference of image
    
  
    lmain = tk.Label(master=root,padx=50,bd=10)
    lmain2 = tk.Label(master=root,bd=10)
    lmain3=tk.Label(master=root,bd=10,fg="#FF0000",bg='black')
    lmain.pack(side=LEFT)
    lmain.place(x=100,y=250)
    lmain3.pack()
    lmain3.place(x=800,y=200)
    lmain2.pack(side=RIGHT)
    lmain2.place(x=800,y=300)
    lmain.img=img
    heading = Label(root,bg="black",width=1, height=1)
    
    heading.pack() 
    heading2=Label(root,text="Photo to Emoji",pady=20, font=('times new roman',45,'bold'),bg='black',fg='#FF0000')                                 
    
    heading2.pack()
    root.title("Photo To Emoji")            
    app=FullScreenApp(root)
    root['bg']='black'
    exitbutton = Button(root, text='QUIT',command=root.destroy,font=('times new roman',15,'bold')).place(x=680,y=630)
    show_vid()
    show_vid2()
    root.mainloop()
 

frame1 is not none
[[155  99  99  99]]
1/1 [==============================] - 0s 53ms/step
6 6 show_text
image/surpriced.png
[[[241 240 236]
  [241 240 236]
  [241 240 236]
  ...
  [241 240 236]
  [241 240 236]
  [241 240 236]]

 [[241 240 236]
  [241 240 236]
  [241 240 236]
  ...
  [241 240 236]
  [241 240 236]
  [241 240 236]]

 [[241 240 236]
  [241 240 236]
  [241 240 236]
  ...
  [241 240 236]
  [241 240 236]
  [241 240 236]]

 ...

 [[241 240 236]
  [241 240 236]
  [241 240 236]
  ...
  [241 240 236]
  [241 240 236]
  [241 240 236]]

 [[241 240 236]
  [241 240 236]
  [241 240 236]
  ...
  [241 240 236]
  [241 240 236]
  [241 240 236]]

 [[241 240 236]
  [241 240 236]
  [241 240 236]
  ...
  [241 240 236]
  [241 240 236]
  [241 240 236]]]
frame2 is not none
image/surpriced.png
[[[241 240 236]
  [241 240 236]
  [241 240 236]
  ...
  [241 240 236]
  [241 240 236]
  [241 240 236]]

 [[241 240 236]
  [241 240 236]
  [241 240 236]
  ...
  [241 240 236]
  [241 240 236]
  [241 240 236]]